
# Movimiento de órdenes de FarmAhorra de Bronze a Silver

## 0. Prerequisitos

In [0]:
from pyspark.sql import functions as F, types as T
from delta.tables import DeltaTable

BRONZE = "farma.bronze.orders"
SILVER = "farma.silver.orders"

## 1. Contexto Unity-Catalog

In [0]:
%sql
USE CATALOG farma;
CREATE SCHEMA IF NOT EXISTS silver;
USE SCHEMA silver;

## 2. Carga desde Bronze y limpieza

In [0]:
b = spark.table(BRONZE)

silver_df = (
    b
    # Derivados de order_datr y order_hour
    .withColumn("order_date", F.to_date("confirmed_at"))
    .withColumn("order_hour", F.date_format("confirmed_at", "HH").cast("int"))
    # Filtro: confirmed_at no nulo
    .where(F.col("confirmed_at").isNotNull())
    # Quita columnas innecesarias
    .drop("farmacia_order_id", "line_total", "items_count")
    # Ordena columnas para lectura más cómoda
    .select(
        "mongo_id",
        "external_order_id","id_farmacia","client_id",
        "discount_pct","subtotal","discount","total",
        "confirmed_at","order_date","order_hour",
        "package_ndc_11","quantity","unit_price","descripcion","generic_name",
        "source","_load_ts"
    )
)

# Upsert por mongo_id
if spark.catalog.tableExists(SILVER):
    DeltaTable.forName(spark, SILVER).alias("t") \
      .merge(silver_df.alias("s"), "t.mongo_id = s.mongo_id") \
      .whenMatchedUpdateAll() \
      .whenNotMatchedInsertAll() \
      .execute()
else:
    silver_df.write.format("delta").saveAsTable(SILVER)

print("Silver actualizado →", SILVER, "| filas:", spark.table(SILVER).count())

Silver actualizado → farma.silver.orders | filas: 400


## 3. Verificaciones

In [0]:
%sql
SELECT COUNT(*) AS n FROM farma.silver.orders;

SELECT mongo_id, order_date, order_hour, total
FROM farma.silver.orders
ORDER BY confirmed_at DESC
LIMIT 10;

mongo_id,order_date,order_hour,total
68c3ac5864d6cb878be4b37c,2025-09-12,5,2.04
68c3ac5764d6cb878be4b37b,2025-09-12,5,1469.46
68c3ac5664d6cb878be4b37a,2025-09-12,5,74.78
68c3ac5664d6cb878be4b379,2025-09-12,5,63.08
68c3ac5564d6cb878be4b378,2025-09-12,5,4.57
68c3ac5464d6cb878be4b377,2025-09-12,5,100.13
68c3ac5464d6cb878be4b376,2025-09-12,5,149.74
68c3ac5364d6cb878be4b375,2025-09-12,5,19.76
68c3ac5264d6cb878be4b374,2025-09-12,5,13.21
68c3ac5264d6cb878be4b373,2025-09-12,5,46.15
